# Creating a Functional Annotation Back-end Database for Chromosome 22 While Conserving Memory: A Dask-Focused Approach

To create this database, almost 40 files need to be used. Most of these are  large, with some exceeding 70+ GB. To prevent memory overload, the Dask python library will be used. By "lazily" loading in the data, Dask will greatly reduce the strain placed on our computer's RAM, allowing us to perform join functions and to manipulate the data more easily. After the merging is complete, we can convert and download this data to CSV.

Our first step is to import some useful packages and set our working directory:

In [1]:
import os
import sys
import dask.dataframe as dd
import pandas as pd
import numpy as np
import csv

print("wd"
     , os.getcwd())

os.chdir('/Volumes/HZU/CADD/hg19')
print("wd"
     , os.getcwd())

wd /Users/patrickhallaert
wd /Volumes/HZU/CADD/hg19


We can then lazily load-in our 39 files using the dask.dataframe.read_csv() function. We will also drop duplicate and unecessary columns from each file, and all column headers to data that does not contain any.

In [2]:
# Master File: The initial dataset to which every other functional annotation file will be merged upon: The CADD chr22 file.
ch22 = dd.read_csv("/Volumes/HZU/CADD/hg19/hg19_CADD_SNVs_inclAnno_chr22.txt", sep='\t', dtype = "str")

In [3]:
# Functional Annotation 1: ClinVar

clinvar = dd.read_csv('/Volumes/HZU/CADD/hg19/clinvar_20220528.txt', comment='#', sep='\t', dtype="str")
clinvar.columns = ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]

# Functional Annotation 2: Eigen

eigen = dd.read_csv("/Volumes/HZU/everyheaders_hg19_Eigen22.txt",sep='\t', on_bad_lines='skip', dtype="str")

# Functional Annotation 3: dbnsfp33a

dbnsfp33a = dd.read_csv("/Volumes/HZU/humandb/everyheader_hg19_dbnsfp33a22.txt",sep='\t', on_bad_lines='skip' , dtype='str')

# Functional Annotation 4: coding FATHMM

c_fathmm = dd.read_csv("/Volumes/HZU/humandb/hg19_fathmm_xf_codingch22headers.txt",sep='\t', dtype = 'str')
c_fathmm = c_fathmm.drop(columns="position")

# Functional Annotation 5: GNOMAD (exome)

exomegnomad = dd.read_csv("/Volumes/HZU/humandb/headers_hg19_gnomad211_exomech22.txt",sep='\t', dtype = 'str')

# Functional Annotation 6: ABRAOM

abraom = dd.read_csv("/Volumes/HZU/humandb/headers_hg19_abraom_ch22.txt",sep='\t', dtype = 'str')

# Functional Annotation 7: AFR Sites

AFR = dd.read_csv('/Volumes/HZU/humandb/hg19_AFR.sites.2015_08_ch22.txt', comment='#', sep='\t' , dtype='str')
AFR.columns = ["#chr", "start", "ref", "alt", "AFR_exome_allele_frequency", "rsID"]

# Functional Annotation 8: ALL Sites

ALL = dd.read_csv('/Volumes/HZU/humandb/hg19_ALL.sites.2015_08_ch22.txt', comment='#', sep='\t' , dtype='str')
ALL.columns = ["#chr", "start", "ref", "alt", "ALL_exome_allele_frequency", "rsID"]

# Functional Annotation 9: AMR Sites

AMR = dd.read_csv('/Volumes/HZU/humandb/hg19_AMR.sites.2015_08_ch22.txt', comment='#', sep='\t' , dtype='str')
AMR.columns = ["#chr", "start", "ref", "alt", "AMR_exome_allele_frequency", "rsID"]

# Functional Annotation 10: EAS Sites

EAS = dd.read_csv('/Volumes/HZU/humandb/hg19_EAS.sites.2015_08_ch22.txt', comment='#', sep='\t' , dtype='str')
EAS.columns = ["#chr", "start", "ref", "alt", "EAS_exome_allele_frequency", "rsID"]

# Functional Annotation 11: EUR Sites

EUR = dd.read_csv("/Volumes/HZU/humandb/ hg19_EUR.sites.2015_08_ch22.txt", sep='\t', dtype = 'str')
EUR.columns = ["#CHROM", "POS", "REF", "ALT", "EUR_exome_allele_frequency", "rsID"]

# Functional Annotation 12: SAS Sites

sas = dd.read_csv("/Volumes/HZU/humandb/hg19_SAS.sites.2015_08_ch22.txt", sep='\t', dtype = 'str')
sas.columns = ["#Chrom", "Pos", "Ref", "Alt", "SAS", "rsID"]
sas = sas.drop(columns=["rsID"])


# Functional Annotation 13: cg46 Sites

cg46 = dd.read_csv("/Volumes/HZU/humandb/hg19_cg46.txt", sep='\t', dtype = 'str')
cg46.columns = ["#Chrom", "position", "Position2", "Ref", "Alt", "cg46"]

# Functional Annotation 14: Cosmic 70

cosmic70 = dd.read_csv("/Volumes/HZU/humandb/hg19_cosmic70.txt", sep='\t', dtype = 'str')
cosmic70.columns = ["#Chrom", "Position", "Position2", "Ref", "Alt", "Cosmic70"]
cosmic70 = cosmic70.drop(columns=['Position2'])

# Functional Annotation 15: dbscsnv11

col_list1 = ["#Chr", "Start", "Ref", "Alt", "dbscSNV_ADA_SCORE", "dbscSNV_RF_SCORE"]
dbscsnv11 = pd.read_csv("/Volumes/HZU/humandb/hg19_dbscsnv11.txt", usecols=col_list1, sep='\t', dtype = 'str')

# Functional Annotation 16: Exac03

col_list2 = ["#Chr", "Start", "Ref", "Alt", "ExAC_ALL", "ExAC_AFR", "ExAC_AMR", "ExAC_EAS", "ExAC_FIN", "ExAC_NFE", "ExAC_OTH", "ExAC_SAS"]
exac03 = dd.read_csv("/Volumes/HZU/humandb/hg19_exac03.txt", usecols=col_list2, sep='\t', dtype = 'str')

# Functional Annotation 17: GallSNP

GallSNP = dd.read_csv("/Volumes/HZU/humandb/hg19_GeuvadisFDR5allSNP.txt", sep='\t', dtype = 'str')

# Functional Annotation 18: GME

gme = dd.read_csv("/Volumes/HZU/humandb/hg19_gme.txt", sep='\t', dtype = 'str')

# Functional Annotation 19: hrcr1

hrcr1 = dd.read_csv("/Volumes/HZU/humandb/hg19_hrcr1_ch22.txt", sep='\t', dtype = 'str')
hrcr1.columns = ["#Chr", "Start", "End", "Ref", "Alt", "HRC_AF", "HRC_AC", "HRC_AN", "HRC_non1000G_AF", "HRC_non1000G_AC", "HRC_non1000G_AN"]
hrcr1 = hrcr1.drop(columns=["End"])

# Functional Annotation 20: icgc21

icgc21 = dd.read_csv("/Volumes/HZU/humandb/hg19_icgc21_ch22.txt", sep='\t', dtype = 'str')
icgc21.columns = ["#Chr", "Start", "End", "Ref", "Alt", "ICGC_Id", "ICGC_Occurrence"]

# Functional Annotation 21: intervar

intervar = dd.read_csv("/Volumes/HZU/humandb/hg19_intervar_20180118_ch22.txt", sep='\t', dtype = 'str')
intervar.columns = ["#Chr", "Start", "End", "Ref", "Alt", "InterVar_automated", "PVS1", "PS1", "PS2", "PS3", "PS4", "PM1", "PM2", "PM3", "PM4", "PM5", "PM6", "PP1", "PP2", "PP3", "PP4", "PP5", "BA1", "BS1", "BS2", "BS3", "BS4", "BP1", "BP2", "BP3", "BP4", "BP5", "BP6", "BP7"]

# Functional Annotation 22: nci60

nci60 = dd.read_csv("/Volumes/HZU/humandb/hg19_nci60.txt", sep='\t', dtype = 'str')
nci60.columns = ["#Chr", "Start", "End", "Ref", "Alt", "nci60"]

# Functional Annotation 23: Kaviar

kaviar = dd.read_csv("/Volumes/HZU/humandb/hg19_kaviar_20150923_ch22.txt", sep='\t', dtype = 'str')
kaviar.columns = ["#Chr", "Start", "End", "Ref", "Alt", "Kaviar_AF", "Kaviar_AC", "Kaviar_AN"]

# Functional Annotation 24: ljb23 FATHMM

lfathmm = dd.read_csv("/Volumes/HZU/humandb/hg19_ljb23_fathmm_ch22.txt", sep='\t', dtype = 'str')
lfathmm.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "LJB23_FATHMM_score", "LJB23_FATHMM_score_converted", "LJB23_FATHMM_pred"]
lfathmm = lfathmm.drop(columns=["Pos2"])

# Functional Annotation 25: PhyloP

phylop = dd.read_csv("/Volumes/HZU/humandb/hg19_ljb23_phylop_ch22.txt", sep='\t', dtype = 'str')
phylop.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "LJB23_PhyloP"]
phylop = phylop.drop(columns=["Pos2"])

# Functional Annotation 26: ljb23 SIFT

sift = dd.read_csv("/Volumes/HZU/humandb/hg19_ljb23_sift_ch22.txt", sep='\t', dtype = 'str')
sift.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "LJB23_SIFT_score", "LJB23_SIFT_score_converted", "LJB23_SIFT_pred"]
sift = sift.drop(columns=["Pos2"])

# Functional Annotation 27: ljb23 SIPHY

siphy = dd.read_csv("/Volumes/HZU/humandb/hg19_ljb23_siphy_ch22.txt", sep='\t', dtype = 'str')
siphy.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "LJB23_SiPhy"]
siphy = siphy.drop(columns=["Pos2"])

# Functional Annotation 28: MCAP

mcap = dd.read_csv("/Volumes/HZU/humandb/hg19_mcap_ch22.txt", sep='\t', dtype = 'str')
mcap.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "MCAP_score"]
mcap = mcap.drop(columns=["Pos2"])

# Functional Annotation 29: PopFreq All

popfreqall = dd.read_csv("/Volumes/HZU/humandb/hg19_popfreq_all_20150413_ch22.txt", sep='\t', dtype = 'str')
popfreqall.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "PopFreqMax", "1000G_ALL", "1000G_AFR", "1000G_AMR", "1000G_EAS", "1000G_EUR", "1000G_SAS", "ExAC_AL", "ExAC_AFR", "ExAC_AMR", "ExAC_EAS", "ExAC_FIN", "ExAC_NFE", "ExAC_OTE", "ExAC_SAS", "ESP6500siv2_ALL", "ESP6500siv2_AA", "ESP6500siv2_EA", "CG46"]
popfreqall = popfreqall.drop(columns=["Pos2"])

# Functional Annotation 30: Regsnpintron

regsnpintron = dd.read_csv("/Volumes/HZU/humandb/hg19_regsnpintron_ch22.txt", sep='\t', dtype = 'str')
regsnpintron.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "regsnp_fpr", "regsnp_disease", "regsnp_splicing_site"]
regsnpintron = regsnpintron.drop(columns=["Pos2"])

# Functional Annotation 31: Revel

revel = dd.read_csv("/Volumes/HZU/humandb/hg19_revel_ch22.txt", sep='\t', dtype = 'str')
revel.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "REVEL"]
revel = revel.drop(columns=["Pos2"])

# Functional Annotation 32: dbnsfp31a InterPro

interpro = dd.read_csv("/Volumes/HZU/humandb/hg19_dbnsfp31a_interpro_ch22.txt", sep='\t', dtype = 'str')
interpro.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "Interpro_domain"]
interpro = interpro.drop(columns=["Pos2"])

# Functional Annotation 33: esp6500siv2_aa

aa = dd.read_csv("/Volumes/HZU/humandb/hg19_esp6500siv2_aa_ch22.txt", sep='\t', dtype = 'str')
aa.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "esp6500_aa", "rsID"]
aa = aa.drop(columns=["Pos2", "rsID"])

# Functional Annotation 34: esp6500siv2_all

espall = pd.read_csv("/Volumes/HZU/humandb/hg19_esp6500siv2_all_ch22.txt", sep='\t', dtype = 'str')
espall.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "esp6500_all", "rsID"]
espall = espall.drop(columns=["Pos2", "rsID"])

# Functional Annotation 35: esp6500siv2_ea

ea = pd.read_csv("/Volumes/HZU/humandb/hg19_esp6500siv2_ea_ch22.txt", sep='\t', dtype = 'str')
ea.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "esp6500_ea", "rsID"]
ea = ea.drop(columns=["Pos2", "rsID"])

# Functional Annotation 36: avsnp150

av = pd.read_csv("/Volumes/HZU/humandb/hg19_avsnp150_ch22.txt", sep='\t', dtype = 'str')
av.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "avsnp150"]
av = av.drop(columns=["Pos2"])

# Functional Annotation 37: DANN


dann = pd.read_csv("/Volumes/HZU/humandb/hg19_dann_ch22.txt", sep='\t', dtype = 'str')
dann.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "DANN"]
dann = dann.drop(columns=["Pos2"])

# Functional Annotation 38: FATHMM xf noncoding

ncfathmm = pd.read_csv("/Volumes/HZU/humandb/hg19_fathmm_xf_noncoding_ch22.txt", sep='\t', dtype = 'str')
ncfathmm.columns = ["#Chrom", "Pos", "Pos2", "Ref", "Alt", "fathmm_xf_noncoding"]
ncfathmm = ncfathmm.drop(columns=["Pos2"])


Next, we can start merging the data, removing duplicate columns and other issues as we go. We will also run a garbage collector and delete objects that are no longer useful in order to further preserve memory.

In [4]:
### Merging Data ###

# Merging Eigen into chr22 CADD

eigen_added = ch22.merge(
    eigen, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'position', 'ref', 'alt'])
eigen_added = eigen_added.drop(columns=['position.1', '#chr', 'position', 'ref', 'alt'])
eigen_added

# Merging dbnsfp33a

dbnsfp33a_added = eigen_added.merge(
    dbnsfp33a, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
dbnsfp33a_added = dbnsfp33a_added.drop(columns=['#chr', 'start', 'ref', 'alt'])
dbnsfp33a_added

# Merging ClinVar

clinvar_added = dbnsfp33a_added.merge(
    clinvar, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#CHROM', 'POS', 'REF', 'ALT'])
clinvar_added = clinvar_added.drop(columns=['#CHROM', "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])

# Merging FATHMM (Coding)

cfathmm_added = clinvar_added.merge(
    c_fathmm, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# Merging GNOMAD (Exome)
    
exomegnomad_added = cfathmm_added.merge(
    exomegnomad, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
exomegnomad_added = exomegnomad_added.drop(columns=['#chr', "start", "ref", "alt", "end_x", "end_y"])

In [5]:
# Merging ABRAOM
    
abraom_added = exomegnomad_added.merge(
    abraom, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
abraom_added = abraom_added.drop(columns=['#chr', "start", "ref", "alt"])

# Merging ALL Sites

ALL_added = abraom_added.merge(
    ALL, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
ALL_added = ALL_added.drop(columns=['#chr', "start", "ref", "alt", "rsID"])

# Merging AFR Sites

AFR_added = ALL_added.merge(
    AFR, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
AFR_added = AFR_added.drop(columns=['#chr', "start", "ref", "alt", "rsID"])

# Merging EAS Sites
    
EAS_added = AFR_added.merge(
    EAS, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
EAS_added = EAS_added.drop(columns=['#chr', "start", "ref", "alt", "rsID"])

# Merging AMR Sites
    
AMR_added = EAS_added.merge(
    AMR, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'start', 'ref', 'alt'])
AMR_added = AMR_added.drop(columns=['#chr', "start", "ref", "alt", "rsID"])

In [6]:
# Merging cg46
    
cg46_added = AMR_added.merge(
    cg46, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chrom', 'position', 'Ref', 'Alt'])
cg46_added = cg46_added.drop(columns=['position', 'end'])

# Merging Cosmic70 
    
cosmic70_added = cg46_added.merge(
    cosmic70, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chrom', 'Position', 'Ref', 'Alt'])
cosmic70_added = cosmic70_added.drop(columns=['Position2', 'Position'])
    
# Merging dbscsnv11
    
db_added = cosmic70_added.merge(
    dbscsnv11, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
db_added = db_added.drop(columns=['#Chr', 'Start'])

# Merging Exac03
    
exac03_added = db_added.merge(
    exac03, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
exac03_added = exac03_added.drop(columns=['#Chr', 'Start'])

# Merging EUR Sites
    
EUR_added = exac03_added.merge(
    EUR, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#CHROM', 'POS', 'REF', 'ALT'])
EUR_added = EUR_added.drop(columns=['#CHROM', "POS", "REF", "ALT"])

In [7]:
# Merging GallSNP

GallSNP_added = EUR_added.merge(
    GallSNP, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#chr', 'pos', 'ref', 'alt'])
    
# GME Added

GME_added = GallSNP_added.merge(
    gme, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])

# hrcr1 Added

hrcr1_added = GME_added.merge(
    hrcr1, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
hrcr1_added = hrcr1_added.drop(columns=['#Chr_x', '#Chr_y', 'Start_y', 'Start_x'])
    
# icgc21 Added

icgc21_added = hrcr1_added.merge(
    icgc21, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
icgc21_added = icgc21_added.drop(columns=['#Chr', "Start", "End_y"])

# Intervar Added

intervar_added = icgc21_added.merge(
    intervar, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
intervar_added = intervar_added.drop(columns=["End_x", "#Chr", "Start", "End", "rsID", "#chr", "pos", "pos.1", "ref", "alt"])

In [8]:
# nci60 Added

nci60_added = intervar_added.merge(
    nci60, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
nci60_added = nci60_added.drop(columns=["#Chr", "Start", "End", '#Chr'])

# Kaviar Added
    
kaviar_added = nci60_added.merge(
    kaviar, 
    how="left", 
    left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#Chr', 'Start', 'Ref', 'Alt'])
kaviar_added = kaviar_added.drop(columns=["#Chr", "Start", "End"])

# LFATHMM added
    
lfathmm_added = kaviar_added.merge(
    lfathmm, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# PhyloP Added
    
phylop_added = lfathmm_added.merge(
    phylop, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
    
# SIFT Added
 
sift_added = phylop_added.merge(
    sift, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

In [9]:
# SIPHY Added
    
siphy_added = sift_added.merge(
    siphy, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# MCAP Added

mcap_added = siphy_added.merge(
    mcap, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
    
# PopFreqAll Added
    
popfreqall_added = mcap_added.merge(
    popfreqall, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
popfreqall_added = popfreqall_added.drop(columns=["ExAC_AFR_x", "ExAC_AMR_x", "ExAC_EAS_x", "ExAC_FIN_x", "ExAC_NFE_x", "ExAC_SAS_x", "ExAC_AFR_y", "ExAC_AMR_y", "ExAC_EAS_y", "ExAC_FIN_y", "ExAC_NFE_y", "ExAC_SAS_y"])

# Regsnpintron Added
    
regsnpintron_added = popfreqall_added.merge(
    regsnpintron, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
    
# Revel Added
    
revel_added = regsnpintron_added.merge(
    revel, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

In [10]:
# SAS Sites Added
    
SAS_added = revel_added.merge(
    sas, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# InterPro Added
    
interpro_added = SAS_added.merge(
    interpro, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
    
# Espall Added
    
espall_added = interpro_added.merge(
    espall, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# EA Sites Added

ea_added = espall_added.merge(
    ea, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
    
# AV Sites Added

av_added = ea_added.merge(
    av, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

In [11]:
# DANN Added
    
dann_added = av_added.merge(
    dann, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])

# FATHMM (Non-Coding) Added

Chr22_Database = dann_added.merge(
    ncfathmm, 
    how="left", 
    on=['#Chrom', 'Pos', 'Ref', 'Alt'])
Chr22_Database = Chr22_Database.drop(columns=["Consequence_x", "Consequence_y", "Eigen-raw_x", "Eigen-raw_y", "Eigen-PC-raw_x", "Eigen-PC-raw_y"])

Let us take a sneak peak at our "lazy" final product: a functional annotation database for the entirety of Chromosome 22.

In [16]:
Chr22_Database

,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,ConsScore,ConsDetail,GC,CpG,motifECount,motifEName,motifEHIPos,motifEScoreChng,oAA,nAA,GeneID,FeatureID,GeneName,CCDS,Intron,Exon,cDNApos,relcDNApos,CDSpos,relCDSpos,protPos,relProtPos,Domain,Dst2Splice,Dst2SplType,minDistTSS,minDistTSE,SIFTcat,SIFTval,PolyPhenCat,PolyPhenVal,priPhCons,mamPhCons,verPhCons,priPhyloP,mamPhyloP,verPhyloP,bStatistic,targetScan,mirSVR-Score,mirSVR-E,mirSVR-Aln,cHmmTssA,cHmmTssAFlnk,cHmmTxFlnk,cHmmTx,cHmmTxWk,cHmmEnhG,cHmmEnh,cHmmZnfRpts,cHmmHet,cHmmTssBiv,cHmmBivFlnk,cHmmEnhBiv,cHmmReprPC,cHmmReprPCWk,cHmmQuies,GerpRS,GerpRSpval,GerpN,GerpS,TFBS,TFBSPeaks,TFBSPeaksMax,tOverlapMotifs,motifDist,Segway,EncH3K27Ac,EncH3K4Me1,EncH3K4Me3,EncExp,EncNucleo,EncOCC,EncOCCombPVal,EncOCDNasePVal,EncOCFairePVal,EncOCpolIIPVal,EncOCctcfPVal,EncOCmycPVal,EncOCDNaseSig,EncOCFaireSig,EncOCpolIISig,EncOCctcfSig,EncOCmycSig,Grantham,SpliceAI-acc-gain,SpliceAI-acc-loss,SpliceAI-don-gain,SpliceAI-don-loss,MMSp_acceptorIntron,MMSp_acceptor,MMSp_exon,MMSp_donor,MMSp_donorIntron,Dist2Mutation,Freq100bp,Rare100bp,Sngl100bp,Freq1000bp,Rare1000bp,Sngl1000bp,Freq10000bp,Rare10000bp,Sngl10000bp,dbscSNV-ada_score,dbscSNV-rf_score,RawScore,PHRED,SIFT,PolyPhenDIV,PolyPhenVar,MA,GERP_NR,GERP_RS,PhyloPri,PhyloPla,PhyloVer,PhastPri,PhastPla,PhastVe,Eigen-phred,Eigen-PC-phred,SIFT_score,SIFT_converted_rankscore,SIFT_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationAssessor_score,MutationAssessor_score_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST3_score,VEST3_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,M-CAP_score,M-CAP_rankscore,M-CAP_pred,CADD_raw,CADD_raw_rankscore,CADD_phred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,Eigen_coding_or_noncoding,GenoCanyon_score,GenoCanyon_score_rankscore,integrated_fitCons_score,integrated_fitCons_score_rankscore,integrated_confidence_value,GERP++_RS,GERP++_RS_rankscore,phyloP100way_vertebrate,phyloP100way_vertebrate_rankscore,phyloP20way_mammalian,phyloP20way_mammalian_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian,phastCons20way_mammalian_rankscore,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,Interpro_domain_x,GTEx_V6_gene,GTEx_V6_tissue,FATHMM_XF_coding,AF,AF_popmax,AF_male,AF_female,AF_raw,AF_afr,AF_sas,AF_amr,AF_eas,AF_nfe,AF_fin,AF_asj,AF_oth,non_topmed_AF_popmax,non_neuro_AF_popmax,non_cancer_AF_popmax,controls_AF_popmax,abraom_freq,abraom_filter,abraom_cegh_filter,ALL_exome_allele_frequency,AFR_exome_allele_frequency,EAS_exome_allele_frequency,AMR_exome_allele_frequency,cg46,Cosmic70,dbscSNV_ADA_SCORE,dbscSNV_RF_SCORE,ExAC_ALL,ExAC_OTH,EUR_exome_allele_frequency,Geuvadis_eQTL_target_gene,GME_AF,GME_NWA,GME_NEA,GME_AP,GME_Israel,GME_SD,GME_TP,GME_CA,HRC_AF,HRC_AC,HRC_AN,HRC_non1000G_AF,HRC_non1000G_AC,HRC_non1000G_AN,ICGC_Id,ICGC_Occurrence,InterVar_automated,PVS1,PS1,PS2,PS3,PS4,PM1,PM2,PM3,PM4,PM5,PM6,PP1,PP2,PP3,PP4,PP5,BA1,BS1,BS2,BS3,BS4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,nci60,Kaviar_AF,Kaviar_AC,Kaviar_AN,LJB23_FATHMM_score,LJB23_FATHMM_score_converted,LJB23_FATHMM_pred,LJB23_PhyloP,LJB23_SIFT_score,LJB23_SIFT_score_converted,LJB23_SIFT_pred,LJB23_SiPhy,MCAP_score,PopFreqMax,1000G_ALL,1000G_AFR,1000G_AMR,1000G_EAS,1000G_EUR,1000G_SAS,ExAC_AL,ExAC_OTE,ESP6500siv2_ALL,ESP6500siv2_AA,ESP6500siv2_EA,CG46,regsnp_fpr,regsnp_disease,regsnp_splicing_site,REVEL,SAS,Interpro_domain_y,esp6500_all,esp6500_ea,avsnp150,DANN,fathmm_xf_noncoding
npartitions=1191,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Now that the database appears correct, let's export it to CSV as one file so we can use it however we please.

In [ ]:
Chr22_Database.to_csv('/Volumes/HZU/Chr22Database.csv', index=False, single_file=True)

This should make the file more easily readable to the human eye; it can also be converted to other file types.